In [1]:
import tensorflow_io as tfio
import tensorflow as tf

2023-12-03 10:13:30.548053: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 10:13:30.611876: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 10:13:30.916851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:13:30.916917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:13:30.964380: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
tfio.__version__

'0.34.0'

In [2]:
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
NUM_COLUMNS = 2

online_train_ds = tfio.experimental.streaming.KafkaBatchIODataset(
    topics=["train"],
    group_id="cgonline",
    servers="127.0.0.1:9092",
    stream_timeout=10000, # in milliseconds, to block indefinitely,set it -1
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

def decode_kafka_online_item(raw_message, raw_key):
    message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
    key = tf.strings.to_number(raw_key)
    return (message, key)
 
for mini_ds in online_train_ds:
    mini_ds = mini_ds.shuffle(buffer_size=32)
    mini_ds = mini_ds.map(decode_kafka_online_item)
    mini_ds = mini_ds.batch(32)
    # if len(mini_ds) > 0:
    #     model.fit(mini_ds, epochs=3)
    print(type(mini_ds))
    print(mini_ds.shape)
    print(mini_ds.take(1))
    break
    

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['/com.docker.devenvironments.code/.venv/lib/python3.11/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/com.docker.devenvironments.code/.venv/lib/python3.11/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrINS5_4core11RefCountPtrIS1_EEEEvEE']